In [60]:
# Import required libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from datetime import timedelta
from tqdm import tqdm
sns.set()
tf.compat.v1.random.set_random_seed(1234)

In [63]:
#import OHLC data
path = Path('./tsla.csv')
tsla_df = pd.read_csv(path, index_col='Date', infer_datetime_format=True, parse_dates=True)

# drop empty column
tsla_df.drop(columns=['Down'], inplace=True)
# drop null values
tsla_df.dropna(inplace=True)
# calc actual returns
tsla_df['actual returns'] = tsla_df['Close'].pct_change()

# drop nulls
tsla_df.dropna(inplace=True)

# display dataframe
tsla_df.drop(columns=['RSI(14)', 'actual returns'], inplace=True)


In [ ]:
# # seperate trade criteria to new dataframe
# trading_df = tsla_df[['Date', 'RSI(14)']]

# create signal
tsla_df['trading signal'] = 0.0
tsla_df


# buy signal
tsla_df.loc[(tsla_df["actual returns"] > 0), "signal"] = 1


# sell signal
tsla_df.loc[(tsla_df["actual returns"] < 0), "signal"] = -1


# create X df
X = tsla_df[['RSI(14)']].shift().dropna().copy()

# create y df
y = tsla_df['signal'].copy()

y

In [ ]:
# Import required libraries
from pandas.tseries.offsets import DateOffset

In [ ]:
# start of training

training_start = X.index.min()

print(training_start)

In [ ]:
# Select the ending period for the training data with an offset of 12 months
training_end = X.index.min() + DateOffset(months=12)

print(training_end)

In [ ]:
# create training sets

X_train = X.loc[training_start:training_end]
y_train = y.loc[training_start:training_end]


# create test sets
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

In [ ]:
# create model
class Model:
    def __init__(
        self,
        learning_rate,
        num_layers,
        size,
        size_layer,
        output_size,
        forget_bias = 0.1,
    ):
        def lstm_cell(size_layer):
            return tf.nn.rnn_cell.GRUCell(size_layer)

        rnn_cells = tf.nn.rnn_cell.MultiRNNCell(
            [lstm_cell(size_layer) for _ in range(num_layers)],
            state_is_tuple = False,
        )
        self.X = tf.placeholder(tf.float32, (None, None, size))
        self.Y = tf.placeholder(tf.float32, (None, output_size))
        drop = tf.contrib.rnn.DropoutWrapper(
            rnn_cells, output_keep_prob = forget_bias
        )
        self.hidden_layer = tf.placeholder(
            tf.float32, (None, num_layers * size_layer)
        )
        self.outputs, self.last_state = tf.nn.dynamic_rnn(
            drop, self.X, initial_state = self.hidden_layer, dtype = tf.float32
        )
        self.logits = tf.layers.dense(self.outputs[-1], output_size)
        self.cost = tf.reduce_mean(tf.square(self.Y - self.logits))
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(
            self.cost
        )
        
def calculate_accuracy(real, predict):
    real = np.array(real) + 1
    predict = np.array(predict) + 1
    percentage = 1 - np.sqrt(np.mean(np.square((real - predict) / real)))
    return percentage * 100

def anchor(signal, weight):
    buffer = []
    last = signal[0]
    for i in signal:
        smoothed_val = last * weight + (1 - weight) * i
        buffer.append(smoothed_val)
        last = smoothed_val
    return buffer
